In [139]:
from solana.rpc.async_api import AsyncClient
from solana.keypair import Keypair
from solana.publickey import PublicKey
from solana.transaction import Transaction, TransactionInstruction, AccountMeta
from spl.token.constants import TOKEN_PROGRAM_ID, ASSOCIATED_TOKEN_PROGRAM_ID
from solana.rpc.types import TxOpts
from spl.token.instructions import get_associated_token_address

import base64, base58
import struct
import time
import pandas as pd
import asyncio

In [140]:
on_devnet = True

In [141]:
SYS_PROGRAM_PUBKEY = PublicKey("11111111111111111111111111111111")
TOKEN_METADATA_PROGRAM_ID = PublicKey("metaqbxxUerdq28cj1RbAWkYQm3ybzjb6a8bt518x1s")
RPC_TOKEN = "627c3aed6df38ade602e4623af92"
RPC_mainnet = f"https://extend.mainnet.rpcpool.com/{RPC_TOKEN}"
RPC_devnet = f"https://extend.devnet.rpcpool.com/{RPC_TOKEN}"

PIXEL_OWNER = "MNTRTGTvzmZUDMYuiPg62rY34jHuThGa8jdyTTx58mC"

if on_devnet:
    solana_client = AsyncClient(RPC_devnet)
else:
    solana_client = AsyncClient(RPC_mainnet)

In [142]:
SPACE_PROGRAM_ID = PublicKey("XSPCZghPXkWTWpvrfQ34Szpx3rwmUjsxebRFf5ckbMD")
COLOR_PROGRAM_ID = PublicKey("XCLReS3yMKtcHWJxW8HX8yr6YmY8rwTaS5NUmVk21mM")
RENT_PROGRAM_ID = PublicKey("8xNhq5mgaZNExbkNk9UmPUHGDQb8PsLDdyYZLd1BHvjd")

BASE = PublicKey("XBSEZzB7ojaKgXqfCSpNbPLnuMGk3JVtSKYjXYqg7Pn")

NEIGHBORHOOD_FRAME_BASE_SEED = b"neighborhood_frame_base"
NEIGHBORHOOD_FRAME_POINTER_SEED = b"neighborhood_frame_key"
NEIGHBORHOOD_METADATA_SEED = b"neighborhood_metadata"
SELL_DELEGATE_SEED = b"sell_delegate"
SPACE_METADATA_SEED = b"space_metadata"
RENT_ACCOUNT_SEED = b"rent_account"

sk = list(pd.read_json(f"~/.config/solana/{PIXEL_OWNER}.json").values.flatten())
auth = Keypair.from_secret_key(bytes(sk))

NBDHOOD_SIZE = 200

## Test SetRent ix

In [144]:
async def set_rent(space_x, space_y, price, min_duration, max_duration, max_ts, create):
    tx = Transaction()

    rent_acc = PublicKey.find_program_address([
                bytes(BASE),
                RENT_ACCOUNT_SEED,
                struct.pack("<q",space_x),
                struct.pack("<q",space_y),
              ], program_id = RENT_PROGRAM_ID)[0]

    space_md = PublicKey.find_program_address([
                bytes(BASE),
                SPACE_METADATA_SEED,
                struct.pack("<q",space_x),
                struct.pack("<q",space_y),
            ], program_id = SPACE_PROGRAM_ID)[0]

    data = await solana_client.get_account_info(space_md)
    mint = PublicKey(base64.b64decode(data['result']['value']['data'][0])[1:33])

    ata_acc = get_associated_token_address(auth.public_key, mint)

    keys = [
            AccountMeta(BASE,False,False),
            AccountMeta(rent_acc,False,True),
            AccountMeta(auth.public_key,True,False),
            AccountMeta(ata_acc,False,False),
            AccountMeta(space_md,False,False),
            AccountMeta(SYS_PROGRAM_PUBKEY,False,False),
            ]

    data = struct.pack("<BqqQQQQ?", 0, space_x, space_y, price, min_duration, max_duration, max_ts, create)
   
    tx.add(
        TransactionInstruction(
            keys = keys,
            data = data,
            program_id = RENT_PROGRAM_ID
        )
    )

    signs = [auth]
    txopts = TxOpts(skip_confirmation=False, skip_preflight=True)

    outp = (await solana_client.send_transaction(tx, *signs, opts=txopts))

    rent_acc_data = await solana_client.get_account_info(rent_acc)
    rent_acc_data_bytes = base64.b64decode(rent_acc_data['result']['value']['data'][0])

    price_state = struct.unpack("<Q",rent_acc_data_bytes[1:9])
    min_duration_state = struct.unpack("<Q",rent_acc_data_bytes[9:17])
    max_duration_state = struct.unpack("<Q",rent_acc_data_bytes[17:25])
    max_ts_state = struct.unpack("<Q",rent_acc_data_bytes[25:33])
    lister_state = PublicKey(rent_acc_data_bytes[33:65])
    rent_end_state = struct.unpack("<Q",rent_acc_data_bytes[65:73])
    renter_state = PublicKey(rent_acc_data_bytes[73:105])

    print(price, price_state)
    print(min_duration, min_duration_state)
    print(max_duration, max_duration_state)
    print(max_ts, max_ts_state)
    print(lister_state)
    print(rent_end_state)
    print(renter_state)

    return outp, rent_acc_data_bytes

### List (1,1)

In [165]:
space_x = 1
space_y = 1
price = 1000
min_duration = 31000000
max_duration = int(31000000*1.5)
max_ts = int(time.time() + 2*min_duration)
create = True

outp, data = await set_rent(space_x, space_y, price, min_duration, max_duration, max_ts, create)

1000 (1000,)
31000000 (31000000,)
46500000 (46500000,)
1705129511 (1705129511,)
MNTRTGTvzmZUDMYuiPg62rY34jHuThGa8jdyTTx58mC
(0,)
11111111111111111111111111111111


### Delist (1,1)

In [166]:
space_x = 1
space_y = 1
price = 1000
min_duration = 31000000
max_duration = int(31000000*1.5)
max_ts = int(time.time() + 2*min_duration)
create = False

outp, data = await set_rent(space_x, space_y, price, min_duration, max_duration, max_ts, create)

1000 (0,)
31000000 (0,)
46500000 (0,)
1705129532 (0,)
MNTRTGTvzmZUDMYuiPg62rY34jHuThGa8jdyTTx58mC
(0,)
11111111111111111111111111111111


### Delist (1,0)

In [158]:
space_x = 1
space_y = 0
price = 1000
min_duration = 31000000
max_duration = int(31000000*1.5)
max_ts = int(time.time() + 2*min_duration)
create = False

outp, data = await set_rent(space_x, space_y, price, min_duration, max_duration, max_ts, create)

1000 (0,)
31000000 (0,)
46500000 (0,)
1705129281 (0,)
11111111111111111111111111111111
(0,)
11111111111111111111111111111111


### List (2,1)

In [167]:
space_x = 2
space_y = 1
price = 2000
min_duration = 31000000
max_duration = int(31000000*1.5)
max_ts = int(time.time() + 2*min_duration)
create = True

outp, data = await set_rent(space_x, space_y, price, 20, max_duration, max_ts, create)

2000 (2000,)
20 (20,)
46500000 (46500000,)
1705129550 (1705129550,)
MNTRTGTvzmZUDMYuiPg62rY34jHuThGa8jdyTTx58mC
(0,)
11111111111111111111111111111111


## Test AcceptRent

In [155]:
async def accept_rent(lessee, space_x, space_y, price, rent_time):
    tx = Transaction()

    rent_acc = PublicKey.find_program_address([
                bytes(BASE),
                RENT_ACCOUNT_SEED,
                struct.pack("<q",space_x),
                struct.pack("<q",space_y),
              ], program_id = RENT_PROGRAM_ID)[0]

    space_md = PublicKey.find_program_address([
                bytes(BASE),
                SPACE_METADATA_SEED,
                struct.pack("<q",space_x),
                struct.pack("<q",space_y),
            ], program_id = SPACE_PROGRAM_ID)[0]

    data = await solana_client.get_account_info(space_md)
    mint = PublicKey(base64.b64decode(data['result']['value']['data'][0])[1:33])

    ata_acc = get_associated_token_address(auth.public_key, mint)

    keys = [
            AccountMeta(BASE,False,False),
            AccountMeta(rent_acc,False,True),
            AccountMeta(lessee.public_key,True,True),
            AccountMeta(auth.public_key,False,True),
            AccountMeta(ata_acc,False,False),
            AccountMeta(space_md,False,False),
            AccountMeta(SYS_PROGRAM_PUBKEY,False,False),
            ]

    data = struct.pack("<BqqQQ", 1, space_x, space_y, price, rent_time)
   
    tx.add(
        TransactionInstruction(
            keys = keys,
            data = data,
            program_id = RENT_PROGRAM_ID
        )
    )

    signs = [lessee, auth]
    txopts = TxOpts(skip_confirmation=False, skip_preflight=True)

    outp = (await solana_client.send_transaction(tx, *signs, opts=txopts))

    rent_acc_data = await solana_client.get_account_info(rent_acc)
    rent_acc_data_bytes = base64.b64decode(rent_acc_data['result']['value']['data'][0])

    price_state = struct.unpack("<Q",rent_acc_data_bytes[1:9])
    min_duration_state = struct.unpack("<Q",rent_acc_data_bytes[9:17])
    max_duration_state = struct.unpack("<Q",rent_acc_data_bytes[17:25])
    max_ts_state = struct.unpack("<Q",rent_acc_data_bytes[25:33])
    lister_state = PublicKey(rent_acc_data_bytes[33:65])
    rent_end_state = struct.unpack("<Q",rent_acc_data_bytes[65:73])
    renter_state = PublicKey(rent_acc_data_bytes[73:105])

    print(price, price_state)
    print(min_duration, min_duration_state)
    print(max_duration, max_duration_state)
    print(max_ts, max_ts_state)
    print(lister_state)
    print(rent_end_state)
    print(renter_state)

    return outp, rent_acc_data_bytes

In [156]:
sk = list(pd.read_json(f"~/.config/solana/HoT_devnet.json").values.flatten())
renter = Keypair.from_secret_key(bytes(sk))

### Rent (1,1) (delisted)

In [168]:
space_x = 1
space_y = 1
price = 1000
rent_time = 2

outp, data = await accept_rent(renter, space_x, space_y, price, rent_time)

Transaction error: {'InstructionError': [0, 'InvalidInstructionData']}


1000 (0,)
31000000 (0,)
46500000 (0,)
1705129550 (0,)
MNTRTGTvzmZUDMYuiPg62rY34jHuThGa8jdyTTx58mC
(0,)
11111111111111111111111111111111


### Rent (2,1) but for low price

In [170]:
space_x = 2
space_y = 1
price = 100
rent_time = 200

outp, data = await accept_rent(renter, space_x, space_y, price, rent_time)

Transaction error: {'InstructionError': [0, 'InvalidInstructionData']}


100 (2000,)
31000000 (20,)
46500000 (46500000,)
1705129550 (1705129550,)
MNTRTGTvzmZUDMYuiPg62rY34jHuThGa8jdyTTx58mC
(0,)
11111111111111111111111111111111


### Rent (2,1) but for time < min_duration

In [173]:
space_x = 2
space_y = 1
price = 2000
rent_time = 2

outp, data = await accept_rent(renter, space_x, space_y, price, rent_time)

Transaction error: {'InstructionError': [0, 'InvalidInstructionData']}


2000 (2000,)
31000000 (20,)
46500000 (46500000,)
1705129550 (1705129550,)
MNTRTGTvzmZUDMYuiPg62rY34jHuThGa8jdyTTx58mC
(0,)
11111111111111111111111111111111


### Rent (2,1) but for time > max_duration

In [175]:
space_x = 2
space_y = 1
price = 2000
rent_time = int(31000000*1.5)+1

outp, data = await accept_rent(renter, space_x, space_y, price, rent_time)

Transaction error: {'InstructionError': [0, 'InvalidInstructionData']}


2000 (2000,)
31000000 (20,)
46500000 (46500000,)
1705129550 (1705129550,)
MNTRTGTvzmZUDMYuiPg62rY34jHuThGa8jdyTTx58mC
(0,)
11111111111111111111111111111111


### Rent (2,1) for short time > min_duration--should succeed

In [177]:
space_x = 2
space_y = 1
price = 2000
rent_time = 30

outp, data = await accept_rent(renter, space_x, space_y, price, rent_time)

2000 (2000,)
31000000 (20,)
46500000 (46500000,)
1705129550 (1705129550,)
MNTRTGTvzmZUDMYuiPg62rY34jHuThGa8jdyTTx58mC
(1643129757,)
HoTjGzCA3PErxEeNJHBxfJscr3MgGdfCVVjV5VuJcKYQ


### Rent (0,1) again immediately--should fail if done quickly enough

In [178]:
space_x = 0
space_y = 1
price = 2000
rent_time = 30

outp, data = await accept_rent(renter, space_x, space_y, price, rent_time)

Transaction error: {'InstructionError': [0, 'InvalidInstructionData']}


2000 (2000,)
31000000 (20,)
46500000 (46500000,)
1705129550 (1705129084,)
MNTRTGTvzmZUDMYuiPg62rY34jHuThGa8jdyTTx58mC
(15920292505471328761,)
7nXZV1pJ1qpoC8hZABSMj5PNpYhATqLffSvMRK5r9WMM
